## 1. Imports  
Start by importing all the libraries you need for the rest of the notebook. These include standard utilities like `os`, `json`, and `pandas`, as well as deep learning libraries like PyTorch and visualization tools like `matplotlib`.

Also, import everything needed for **distributed training and data processing with Ray**:
- `ray` and `ray.data` provide the high-level distributed data API.
- `ray.train` gives you `TorchTrainer`, `ScalingConfig`, checkpointing, and metrics reporting.
- `prepare_model` wraps your PyTorch model for multi-worker training with Distributed Data Parallel (DDP).

A few extra helpers like `tqdm` and `train_test_split` round out the list for progress bars and quick offline preprocessing.

In [ ]:
# 00. Runtime setup
import os, sys, subprocess

# Non-secret env var 
os.environ["RAY_TRAIN_V2_ENABLED"] = "1"

# Install Python dependencies 
subprocess.check_call([
    sys.executable, "-m", "pip", "install", "--no-cache-dir",
    "torch==2.8.0",
    "matplotlib==3.10.6",
    "pyarrow==14.0.2",
])

In [ ]:
# 01. Imports

# Standard libraries
import os
import uuid
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import shutil
import tempfile

# PyTorch
import torch
from torch import nn
import torch.nn.functional as F

# Ray
import ray
import ray.data
from ray.train import ScalingConfig, RunConfig, CheckpointConfig, FailureConfig, Checkpoint, get_checkpoint, get_context,  get_dataset_shard, report
from ray.train.torch import TorchTrainer, prepare_model

# Other
from tqdm import tqdm

### 2. Load MovieLens 100K dataset

Download and extract the [MovieLens 100K](https://grouplens.org/datasets/movielens/100k/) dataset, then persist a cleaned copy under `/mnt/cluster_storage/rec_sys_tutorial/raw/` in **two formats**:

- **CSV:** `ratings.csv` (kept for later inference cells).  
- **Parquet dataset:** `ratings_parquet/` as multiple shards (production-style blob store layout) so Ray Data can **stream** reads in parallel without materializing the full dataset.

The output has four columns: `user_id`, `item_id`, `rating`, and `timestamp`.

The MovieLens 100K dataset contains **100,000 ratings** across **943 users** and **1,682 movies** — small enough for quick iteration, yet realistic for demonstrating distributed streaming and training with **Ray Data + Ray Train**.

In [ ]:
# 02. Load MovieLens 100K Dataset and store in /mnt/cluster_storage/ as CSV + Parquet

# Define clean working paths
DATA_URL = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
LOCAL_ZIP = "/mnt/cluster_storage/rec_sys_tutorial/ml-100k.zip"
EXTRACT_DIR = "/mnt/cluster_storage/rec_sys_tutorial/ml-100k"
OUTPUT_CSV = "/mnt/cluster_storage/rec_sys_tutorial/raw/ratings.csv"
PARQUET_DIR = "/mnt/cluster_storage/rec_sys_tutorial/raw/ratings_parquet"

# Ensure target directories exist
os.makedirs("/mnt/cluster_storage/rec_sys_tutorial/raw", exist_ok=True)

# Download only if not already done
if not os.path.exists(LOCAL_ZIP):
    !wget -q $DATA_URL -O $LOCAL_ZIP

# Extract cleanly
if not os.path.exists(EXTRACT_DIR):
    import zipfile
    with zipfile.ZipFile(LOCAL_ZIP, 'r') as zip_ref:
        zip_ref.extractall("/mnt/cluster_storage/rec_sys_tutorial")

# Load raw file
raw_path = os.path.join(EXTRACT_DIR, "u.data")
df = pd.read_csv(raw_path, sep="\t", names=["user_id", "item_id", "rating", "timestamp"])

# Persist CSV (kept for later inference cell that expects CSV)
df.to_csv(OUTPUT_CSV, index=False)

# Persist a Parquet *dataset* (multiple files) to simulate blob storage layout
if os.path.exists(PARQUET_DIR):
    shutil.rmtree(PARQUET_DIR)
os.makedirs(PARQUET_DIR, exist_ok=True)

NUM_PARQUET_SHARDS = 8
for i, shard in enumerate(np.array_split(df, NUM_PARQUET_SHARDS)):
    shard.to_parquet(os.path.join(PARQUET_DIR, f"part-{i:02d}.parquet"), index=False)

print(f"✅ Loaded {len(df):,} ratings → CSV: {OUTPUT_CSV}")
print(f"✅ Wrote Parquet dataset with {NUM_PARQUET_SHARDS} shards → {PARQUET_DIR}")

### 3. Point to Parquet dataset URI

Instead of creating a Ray Dataset from in-memory pandas objects, this tutorial now reads data directly from a **Parquet dataset** stored in persistent cluster storage.

This URI will be used by Ray Data to **stream** Parquet shards efficiently across workers without loading the full dataset into memory.

In [ ]:
# 03. Point to Parquet dataset URI 
DATASET_URI = os.environ.get(
    "RATINGS_PARQUET_URI",
    "/mnt/cluster_storage/rec_sys_tutorial/raw/ratings_parquet",
)

print("Parquet dataset URI:", DATASET_URI)

### 4. Visualize dataset: ratings, users, and items

Before training, visualize the distribution of ratings, user activity, and item popularity.  
These plots serve as a quick sanity check to confirm the dataset loaded correctly and to highlight patterns in user–item interactions:

- **Rating distribution:** shows how often each rating (1–5 stars) occurs, typically skewed toward higher scores.  
- **Ratings per user:** reveals the long-tail behavior where a few users rate many items, while most rate only a few.  
- **Ratings per item:** similarly shows that a handful of popular items receive most of the ratings.

This visualization works with either raw IDs (`user_id`, `item_id`) or encoded indices (`user_idx`, `item_idx`), depending on what’s available in the current DataFrame.


In [ ]:
# 04. Visualize dataset: ratings, user and item activity

# Use encoded indices if present; otherwise fall back to raw IDs
user_col = "user_idx" if "user_idx" in df.columns else "user_id"
item_col = "item_idx" if "item_idx" in df.columns else "item_id"

plt.figure(figsize=(12, 4))

# Rating distribution
plt.subplot(1, 3, 1)
df["rating"].hist(bins=[0.5,1.5,2.5,3.5,4.5,5.5], edgecolor='black')
plt.title("Rating Distribution")
plt.xlabel("Rating"); plt.ylabel("Frequency")

# Number of ratings per user
plt.subplot(1, 3, 2)
df[user_col].value_counts().hist(bins=30, edgecolor='black')
plt.title("Ratings per User")
plt.xlabel("# Ratings"); plt.ylabel("Users")

# Number of ratings per item
plt.subplot(1, 3, 3)
df[item_col].value_counts().hist(bins=30, edgecolor='black')
plt.title("Ratings per Item")
plt.xlabel("# Ratings"); plt.ylabel("Items")

plt.tight_layout()
plt.show()

### 5. Create Ray Dataset from Parquet and encode IDs

Read the MovieLens ratings directly from the **Parquet dataset** using `ray.data.read_parquet()`. This keeps data in a **streaming, non-materialized** form suitable for large-scale distributed processing.

Next, build lightweight **global ID mappings** for users and items on the driver to convert raw `user_id` and `item_id` values into contiguous integer indices (`user_idx`, `item_idx`) required for embedding layers.  
This mapping step materializes only the distinct IDs (a small subset of the data) while keeping the main dataset lazy and scalable.

Finally, apply a `map_batches()` transformation to encode each batch of rows in parallel across the cluster.  
The resulting **Ray Dataset** remains distributed and ready for streaming batches directly into the Ray Train workers.

In [ ]:
# 05. Create Ray Dataset by reading Parquet, then encode IDs via Ray

# Read Parquet dataset directly
ratings_ds = ray.data.read_parquet(DATASET_URI)
print("✅ Parquet dataset loaded (streaming, non-materialized)")
ratings_ds.show(3)

# ---- Build global ID mappings on the driver ----
user_ids = sorted([r["user_id"] for r in ratings_ds.groupby("user_id").count().take_all()])
item_ids = sorted([r["item_id"] for r in ratings_ds.groupby("item_id").count().take_all()])

user2idx = {uid: j for j, uid in enumerate(user_ids)}
item2idx = {iid: j for j, iid in enumerate(item_ids)}

NUM_USERS = len(user2idx)
NUM_ITEMS = len(item2idx)
print(f"Users: {NUM_USERS:,} | Items: {NUM_ITEMS:,}")

# ---- Encode to contiguous indices within Ray (keeps everything distributed) ----
def encode_batch(pdf: pd.DataFrame) -> pd.DataFrame:
    pdf["user_idx"] = pdf["user_id"].map(user2idx).astype("int64")
    pdf["item_idx"] = pdf["item_id"].map(item2idx).astype("int64")
    return pdf[["user_idx", "item_idx", "rating", "timestamp"]]

ratings_ds = ratings_ds.map_batches(encode_batch, batch_format="pandas")
print("✅ Encoded Ray Dataset schema:", ratings_ds.schema())
ratings_ds.show(3)

### 6. Train/validation split using Ray Data  
Next, split the dataset into training and validation sets. First, shuffle the entire Ray Dataset to ensure randomization, then split by row index, using 80% for training and 20% for validation.

This approach is simple and scalable: Ray handles the shuffling and slicing in parallel across blocks. Also, set a fixed seed to ensure the split is reproducible. After you split it, each dataset remains a fully distributed Ray Dataset, ready to stream into workers.

In [ ]:
# 06. Train/val split using Ray Data (lazy, avoids materialization)

TRAIN_FRAC = 0.8
SEED = 42  # for reproducibility

# Block-level shuffle + proportional split (approximate by block, lazy)
train_ds, val_ds = (
    ratings_ds
    .randomize_block_order(seed=SEED)   # lightweight; no row-level materialization
    .split_proportionately([TRAIN_FRAC])  # returns [train, remainder]
)

print("✅ Train/Val Split:")
print(f"  Train → {train_ds.count():,} rows")
print(f"  Val   → {val_ds.count():,} rows")